In [1]:
# Install libraries to google colab environment
!pip install hazm
!pip install stopwords_guilannlp
!pip install -U -q PyDrive

    100% |████████████████████████████████| 317kB 25.4MB/s 
    100% |████████████████████████████████| 1.4MB 17.8MB/s 
    100% |████████████████████████████████| 235kB 30.7MB/s 
  Running setup.py bdist_wheel for nltk ... - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Running setup.py bdist_wheel for libwapiti ... - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [0]:
# Import libraries
import json
import nltk
import numpy as np
# from __future__ import unicode_literals
from hazm import *
import pandas as pd
from collections import OrderedDict
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import collections
import re
from collections import Counter, defaultdict
from stopwords_guilannlp import stopwords_output
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from unidecode import unidecode
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

# Upload Datasets to Colab Environment

## Upload from Google Drive

### Authorize

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Get AsrIran dateset

In [4]:
link = 'https://drive.google.com/open?id=1mfioBj87CdQFWuK08NKb1LpFYb7oq8KE' # The shareable link
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('asriran.jsonl')
# Dataset is now stored in a Pandas Dataframe

1mfioBj87CdQFWuK08NKb1LpFYb7oq8KE


In [5]:
# Import asriran dataset
file = open('asriran.jsonl', 'r')
asriran = file.readlines()


print('There is {} item available'.format(len(asriran)))

There is 15000 item available


### Get FarsNews dataset

In [6]:
link = 'https://drive.google.com/open?id=1PX5zaf4vftkoWMp1ejA6KKqAw1zZx5Vr' # The shareable link
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('farsnews.jsonl')
# Dataset is now stored in a Pandas Dataframe

1PX5zaf4vftkoWMp1ejA6KKqAw1zZx5Vr


In [16]:
# Import farsnews dataset
file = open('farsnews.jsonl', 'r')
farsnews = file.readlines()


print('There is {} item available'.format(len(farsnews)))

There is 15000 item available


##  Json example:
   

```
{
  "_id": "5b4f7280020eb20597f40200",
  "NewsAgency": "AsrIran",
  "newsCode": "621582",
  "newsLink": "http://www.asriran.com/fa/news/621582",
  "date": "تاریخ انتشار:  ۱۴:۴۷ - ۲۷ تير ۱۳۹۷ - 18 July 2018",
  "newsPath": "صفحه نخست » اقتصادی",
  "newsPathLinks": {
    "صفحه نخست": "/fa/archive?service_id=1",
    "اقتصادی": "/fa/archive?service_id=1&amp;cat_id=4"
  },
  "title": "تعداد شعب داخلی شبکه بانکي کاهش یافت",
  "rutitr": "",
  "subtitle": "تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397، به 20598 شعبه ‌رسيد.",
  "body": "تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397 نسبت به پايان اسفند ماه&nbsp; 1396، 0.6 درصد کاهش یافت.به گزارش عصرایران به نقل از بانک مرکزی، تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397، به&nbsp; 20598&nbsp; شعبه &zwnj;رسيد.بر این اساس و در پایان خرداد ماه 1397، تعداد شعب بانک هاي دولتي 8681 شعبه ( 42.1 درصد) ، بانک هاي خصوصي شده 6461 شعبه ( 31.4 درصد) و بانک هاي خصوصي و موسسات اعتباري 5456 شعبه ( 26.5 درصد) است.همچنین تعداد کل شعب ريالي ـ ارزي شبکه بانکي کشور بالغ بر 3787 شعبه، تعداد شعب شهري 19735 و شعب روستايي 863 شعبه است.بر اساس این گزارش تعداد شعب خارج از کشور شبکه بانکي تعداد 48 شعبه و تعداد باجه هاي بانکي کشور در پايان خرداد ماه 1397 تعداد 2274&nbsp; باجه است و213 شعبه بانکی نیز در مناطق آزاد تجاري قرار دارند.بررسی تعداد شعب داخل کشور بانک ها و موسسات اعتباری&nbsp; به تفکیک استانی در پایان خرداد ماه سال 1397 نیز نشان می دهد که استان&zwnj;هاي تهران، اصفــهان و خراسان رضوي به &zwnj;ترتيب با 4049، 1431 و 1318 شعبه داراي بيشترين تعداد شعب هستند.گزارش فصلی &laquo;اطلاعات تعداد شعب بانک ها&nbsp; و موسسات اعتباری&raquo; توسط اداره اطلاعات بانکی بانک مرکزی تهیه و در آن تعداد شعب&nbsp; بانک ها و موسسات اعتباری به تفکیک گروه های بانکی و استانی اعلام می شود.",
  "bodyHtml": " <div align=\"justify\"><img class=\"image_btn\" style=\"margin: 0px 14px;\" title=\"تعداد شعب داخلی شبکه بانکي کاهش یافت\" src=\"http://cdn.asriran.com/files/fa/news/1397/4/27/875553_322.jpg\" alt=\"تعداد شعب داخلی شبکه بانکي کاهش یافت\" width=\"250\" height=\"169\" align=\"left\"/>تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397 نسبت به پايان اسفند ماه&nbsp; 1396، 0.6 درصد کاهش یافت.<br/><br/>به گزارش <span style=\"color: #0000cd;\">عصرایران</span> به نقل از بانک مرکزی، تعداد کل شعب داخل کشور شبکه بانکي در پایان خرداد ماه 1397، به&nbsp; 20598&nbsp; شعبه &zwnj;رسيد.<br/><br/>بر این اساس و در پایان خرداد ماه 1397، تعداد شعب بانک هاي دولتي 8681 شعبه ( 42.1 درصد) ، بانک هاي خصوصي شده 6461 شعبه ( 31.4 درصد) و بانک هاي خصوصي و موسسات اعتباري 5456 شعبه ( 26.5 درصد) است.<br/><br/>همچنین تعداد کل شعب ريالي ـ ارزي شبکه بانکي کشور بالغ بر 3787 شعبه، تعداد شعب شهري 19735 و شعب روستايي 863 شعبه است.<br/><br/><img class=\"image_btn\" style=\"margin: 0px auto; display: block;\" title=\"تعداد شعب داخلی شبکه بانکي کاهش یافت\" src=\"http://cdn.asriran.com/files/fa/news/1397/4/27/875556_701.jpg\" alt=\"تعداد شعب داخلی شبکه بانکي کاهش یافت\" width=\"404\" height=\"300\" align=\"\"/><br/><br/>بر اساس این گزارش تعداد شعب خارج از کشور شبکه بانکي تعداد 48 شعبه و تعداد باجه هاي بانکي کشور در پايان خرداد ماه 1397 تعداد 2274&nbsp; باجه است و213 شعبه بانکی نیز در مناطق آزاد تجاري قرار دارند.<br/><br/>بررسی تعداد شعب داخل کشور بانک ها و موسسات اعتباری&nbsp; به تفکیک استانی در پایان خرداد ماه سال 1397 نیز نشان می دهد که استان&zwnj;هاي تهران، اصفــهان و خراسان رضوي به &zwnj;ترتيب با 4049، 1431 و 1318 شعبه داراي بيشترين تعداد شعب هستند.<br/><br/>گزارش فصلی &laquo;اطلاعات تعداد شعب بانک ها&nbsp; و موسسات اعتباری&raquo; توسط اداره اطلاعات بانکی بانک مرکزی تهیه و در آن تعداد شعب&nbsp; بانک ها و موسسات اعتباری به تفکیک گروه های بانکی و استانی اعلام می شود.</div> <div class=\"wrapper\"/> ",
  "tags": {
    "بانک مرکزی": "/fa/tag/1/بانک مرکزی",
    "شعب داخلی": "/fa/tag/1/شعب داخلی"
  }
}

```



## Load jsons into Dataframe

In [0]:
def asriran_prepare_json(j):
  # clear newsPath
  newsPath = j['newsPath']
  newsPath = newsPath.replace('عصرايران دو » ','')
  newsPath = newsPath.replace('صفحه نخست » ','')
  j['newsPath'] = newsPath
  return j

# For category predictor we need newsPath
raw_data = []
# For agency predictor we need NewsAgency
agency_data = []

for item in asriran:
  try:
    j = json.loads(item)
    asriran_prepare_json(j)
    raw_data.append({'title': j['title'], 'body':j['body'], 'newsPath':j['newsPath'], 'tag':list(j['tags'].keys())[0] })
    agency_data.append({'title': j['title'], 'body':j['body'], 'newsAgency': j['NewsAgency']})
  except:
    pass
  
for item in farsnews:
  try:
    j = json.loads(item)
    agency_data.append({'title': j['title'], 'body':j['body'], 'newsAgency': j['NewsAgency']})
  except:
    pass


In [96]:
category_df = pd.DataFrame(raw_data)
category_df.head()

,body,newsPath,tag,title
0,تهران- ایرنا- رئیس پلیس راه کشور گفت: اکنون در...,اجتماعی,پلیس,جاده های خراسان شمالی و رضوی لغزنده است
1,رئیس فراکسیون مبارزه با مفاسد اقتصادی مجلس شور...,سیاسی,مفاسد اقتصادی,نماینده مجلس: شناسایی 31 نفر از سران مفاسد اقت...
2,سرمربی استقلال در مورد حضور فرهاد مجیدی در کاد...,ورزشی,شفر,واکنش شفر به حضور فرهاد مجیدی در کادرفنی
3,معاون رئیس جمهور آمریکا اعلام کرد که دونالد تر...,بین الملل,آمریکا,مایک پنس: ترامپ در حال بررسی خروج نظامیان آمری...
4,نشریه انگلیسی &laquo;تایمز&raquo; از تابعیت ان...,بین الملل,مسکو,جاسوس آمریکا در روسیه، انگلیسی از آب درآمد


In [97]:
agency_df = pd.DataFrame(agency_data)
agency_df.head()

,body,newsAgency,title
0,تهران- ایرنا- رئیس پلیس راه کشور گفت: اکنون در...,AsrIran,جاده های خراسان شمالی و رضوی لغزنده است
1,رئیس فراکسیون مبارزه با مفاسد اقتصادی مجلس شور...,AsrIran,نماینده مجلس: شناسایی 31 نفر از سران مفاسد اقت...
2,سرمربی استقلال در مورد حضور فرهاد مجیدی در کاد...,AsrIran,واکنش شفر به حضور فرهاد مجیدی در کادرفنی
3,معاون رئیس جمهور آمریکا اعلام کرد که دونالد تر...,AsrIran,مایک پنس: ترامپ در حال بررسی خروج نظامیان آمری...
4,نشریه انگلیسی &laquo;تایمز&raquo; از تابعیت ان...,AsrIran,جاسوس آمریکا در روسیه، انگلیسی از آب درآمد


In [98]:
np.unique(category_df['newsPath'])

array(['آشپزی', 'اجتماعی', 'اقتصادی', 'بین الملل', 'حوادث',
       'خواندنی ها و دیدنی ها', 'داستان کوتاه', 'دانلود', 'روانشناسی',
       'سرگرمی', 'سفر و تفریح', 'سلامت', 'سیاست خارجی', 'سیاسی', 'علمی',
       'عمومی', 'فرهنگی/هنری', 'فناوری و IT', 'ورزشی', 'پیامک'],
      dtype=object)

# Encoding categorical features 

In [0]:
category_le = LabelEncoder()
category_df['newsPath'] = category_le.fit_transform(category_df['newsPath'])

agency_le = LabelEncoder()
agency_df['newsAgency'] = agency_le.fit_transform(agency_df['newsAgency'])

category_x = category_df[['title','body','tag']]

category_y = category_df['newsPath']

agency_x = agency_df[['title','body']]

agency_y = agency_df['newsAgency']

In [90]:
category_x.head()

,title,body,tag
0,جاده های خراسان شمالی و رضوی لغزنده است,تهران- ایرنا- رئیس پلیس راه کشور گفت: اکنون در...,پلیس
1,نماینده مجلس: شناسایی 31 نفر از سران مفاسد اقت...,رئیس فراکسیون مبارزه با مفاسد اقتصادی مجلس شور...,مفاسد اقتصادی
2,واکنش شفر به حضور فرهاد مجیدی در کادرفنی,سرمربی استقلال در مورد حضور فرهاد مجیدی در کاد...,شفر
3,مایک پنس: ترامپ در حال بررسی خروج نظامیان آمری...,معاون رئیس جمهور آمریکا اعلام کرد که دونالد تر...,آمریکا
4,جاسوس آمریکا در روسیه، انگلیسی از آب درآمد,نشریه انگلیسی &laquo;تایمز&raquo; از تابعیت ان...,مسکو


In [91]:
category_y

0         1
1        13
2        18
3         3
4         3
5         3
6         3
7        13
8         1
9        13
10       16
11        1
12       13
13        1
14       16
15        2
16        3
17       13
18        1
19        2
20       13
21       13
22       16
23        1
24        1
25       16
26        1
27        1
28        1
29        3
         ..
14297     1
14298    13
14299    13
14300    13
14301     2
14302    13
14303    13
14304     2
14305     1
14306     3
14307    18
14308     2
14309     3
14310     2
14311     1
14312    13
14313    13
14314    13
14315     3
14316    13
14317    13
14318    13
14319     1
14320     1
14321     2
14322     1
14323    13
14324    13
14325     1
14326    18
Name: newsPath, Length: 14327, dtype: int64

# Split TrainSet and TestSet

In [112]:
# Split for category predictor
x_cat_train, x_cat_test, y_cat_train, y_cat_test = train_test_split(category_x, category_y, test_size=0.20)
print('--Category Predictor--')
print('Train size: ',len(x_cat_train))
print('Test size : ',len(x_cat_test))

--Category Predictor--
Train size:  11461
Test size :  2866


In [24]:
# Split for category predictor
x_ag_train, x_ag_test, y_ag_train, y_ag_test = train_test_split(agency_x, agency_y, test_size=0.20)
print('--Agency Predictor--')
print('Train size: ',len(x_ag_train))
print('Test size : ',len(x_ag_test))

--Agency Predictor--
Train size:  23461
Test size :  5866


# Preprocess data

In [0]:
stop_set = stopwords_output("Persian", "set")
customized_stop_words = ['&zwnj;','zwnj;','&zwnj', '&laquo;', '&raquo;', '&nbsp;', 'ViewAjax', 'asran', 'lse', 'pletely', 'secd', 'vent', 'ventDeult', 'ad_type', 'ads', 'advert', 'adverts', 'ajax', 'also', 'api', 'bull', 'cdn', 'click', 'clip', 'com', 'completely', 'data', 'disable', 'document', 'embed', 'enable', 'fa', 'false', 'feature', 'files', 'find', 'flowplayer', 'for', 'fp', 'function', 'height', 'hellip', 'http', 'if', 'ima', 'ir', 'keyboard', 'laquo', 'ldquo', 'lrm', 'max_ad_duration', 'mp4', 'nbsp', 'ndash', 'news', 'on', 'player_', 'playlist', 'post', 'pre', 'prevent', 'preventDefault', 'raquo', 'ratio', 'rdquo', 'ready', 'return', 'rlm', 'roll', 'sdmax', 'second', 'seeking', 'set', 'sources', 'splash', 'src', 'standardvideo', 'this', 'time', 'true', 'type', 'ui', 'up', 'var', 'via', 'video', 'videoViewAjax', 'width', 'www', 'zwnj']
customized_numbs = ['00', '10', '11', '12', '13', '1397', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '27', '28', '29', '2hzr', '30', '40', '45', '97']
# Remove some redundant words before tokenization and vectorization

for num in customized_numbs:
  stop_set.add(num)

def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False
  
def sentence_preprocess(sentence):
  tokenized = word_tokenize(sentence)
  tokens = []
  for word in tokenized:
    # Remove digits
    # Remove 1 letter words
    for sw in customized_stop_words:
      word = word.replace(sw, '')
    if not (word.isdigit() or isfloat(word) or len(word)<2 or (word in stop_set)):
      tokens.append(word)
  # Return as a string
  tokens = ' '.join(tokens)
  return tokens

def preprocess(data):
  data_list = []
  for sentence in data:
    data_list.append(sentence_preprocess(sentence))
  return data_list

# Vectorizer (Count Vectorzer | TF-IDF Vectorizer)

## Vectorize Category Data

In [26]:
cat_vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,1), min_df=0.005)
# train set
x_cat_train_corpus = x_cat_train['body'].values
x_cat_train_corpus = preprocess(x_cat_train_corpus)
# test set
x_cat_test_corpus = x_cat_test['body'].values
x_cat_test_corpus = preprocess(x_cat_test_corpus)

# Fit vectorizer
cat_vectorizer.fit(x_cat_train_corpus)

# Print vocabulary
cat_vocab = cat_vectorizer.get_feature_names()
print(cat_vocab)
print('Vocab length:', len(cat_vocab))

# Transform vectorizer over train and test set
x_cat_train_vec = cat_vectorizer.transform(x_cat_train_corpus).toarray()
x_cat_test_vec = cat_vectorizer.transform(x_cat_test_corpus).toarray()

['10', '1397', '16', '30', 'CFT', 'Deult', 'FATF', 'asran', 'secd', 'shy', 'آب', 'آباد', 'آبادان', 'آبادی', 'آبان', 'آبانماه', 'آبهای', 'آبگرفتگی', 'آبی', 'آتش', 'آتی', 'آثار', 'آخرین', 'آدرس', 'آدم', 'آذر', 'آذربایجان', 'آذرماه', 'آذری', 'آرا', 'آرام', 'آرامش', 'آرای', 'آرزو', 'آرزوی', 'آرژانتین', 'آزاد', 'آزادی', 'آزار', 'آزمایش', 'آزمایشی', 'آزمون', 'آسان', 'آسانی', 'آستانه', 'آسمان', 'آسوشیتدپرس', 'آسیا', 'آسیایی', 'آسیب', 'آسیبهای', 'آشنا', 'آشنایی', 'آشکار', 'آغاز', 'آفتاب', 'آفریقا', 'آفرین', 'آل', 'آلمان', 'آلمانی', 'آلوده', 'آلودگی', 'آماده', 'آمادگی', 'آمار', 'آمارها', 'آمارهای', 'آماری', 'آمبولانس', 'آمدند', 'آمده_است', 'آمده_بود', 'آمریکا', 'آمریکاست', 'آمریکای', 'آمریکایی', 'آمریکاییها', 'آموز', 'آموزان', 'آموزش', 'آموزشی', 'آمیز', 'آناتولی', 'آنتی', 'آنجایی', 'آنلاین', 'آنهایی', 'آنکارا', 'آهن', 'آوردند', 'آورده_است', 'آورند', 'آوری', 'آژانس', 'آگاه', 'آگاهی', 'آیت', 'آیتالله', 'آینده', 'آیین', 'ائتلاف', 'ابتدا', 'ابتدای', 'ابتدایی', 'ابتلا', 'ابتکار', 'ابر', 'ابراز', 'اب

In [27]:
print('Shape of x_train vector for category predictor: ', x_cat_train_vec.shape)
print('Shape of x_test vector for category predictor: ', x_cat_test_vec.shape)

Shape of x_train vector for category predictor:  (11461, 3934)
Shape of x_test vector for category predictor:  (2866, 3934)


## Vectorize Agency Data

In [28]:
ag_vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,1), min_df=0.005)
# train set
x_ag_train_corpus = x_ag_train['body'].values
x_ag_train_corpus = preprocess(x_ag_train_corpus)
# test set
x_ag_test_corpus = x_ag_test['body'].values
x_ag_test_corpus = preprocess(x_ag_test_corpus)

# Fit vectorizer
ag_vectorizer.fit(x_ag_train_corpus)

# Print vocabulary
ag_vocab = ag_vectorizer.get_feature_names()
print(ag_vocab)
print('Vocab length:', len(ag_vocab))

# Transform vectorizer over train and test set
x_ag_train_vec = ag_vectorizer.transform(x_ag_train_corpus).toarray()
x_ag_test_vec = ag_vectorizer.transform(x_ag_test_corpus).toarray()

['10', '1397', '16', '30', 'CFT', 'Deult', 'FATF', 'asran', 'secd', 'آئین', 'آب', 'آباد', 'آبادان', 'آبادی', 'آبان', 'آبی', 'آتش', 'آتی', 'آثار', 'آثاری', 'آحاد', 'آخرین', 'آدرس', 'آدم', 'آذر', 'آذربایجان', 'آذرماه', 'آذری', 'آرا', 'آرام', 'آرامش', 'آرای', 'آرزو', 'آرزوی', 'آرش', 'آرمان', 'آرژانتین', 'آزاد', 'آزادی', 'آزار', 'آزمایش', 'آزمایشی', 'آزمون', 'آسان', 'آستان', 'آستانه', 'آسمان', 'آسوشیتدپرس', 'آسیا', 'آسیای', 'آسیایی', 'آسیب', 'آسیبهای', 'آشنا', 'آشنایی', 'آشکار', 'آغاز', 'آفریقا', 'آفرین', 'آفرینی', 'آل', 'آلمان', 'آلمانی', 'آلوده', 'آلودگی', 'آماده', 'آمادگی', 'آمار', 'آمارها', 'آمارهای', 'آماری', 'آمبولانس', 'آمدند', 'آمده_است', 'آمده_بود', 'آمریکا', 'آمریکاست', 'آمریکای', 'آمریکایی', 'آموز', 'آموزان', 'آموزش', 'آموزشی', 'آموزی', 'آمیز', 'آن', 'آناتولی', 'آنتن', 'آنجایی', 'آنلاین', 'آنهایی', 'آنکارا', 'آهن', 'آور', 'آورد', 'آوردند', 'آورده_است', 'آورند', 'آوری', 'آوریل', 'آژانس', 'آگاه', 'آگاهی', 'آیت', 'آیتالله', 'آید', 'آیند', 'آینده', 'آیین', 'ائتلاف', 'ائمه', 'ابتدا',

In [29]:
print('Shape of x_train vector for agency predictor: ', x_ag_train_vec.shape)
print('Shape of x_test vector for agency predictor: ', x_ag_test_vec.shape)

Shape of x_train vector for agency predictor:  (23461, 4075)
Shape of x_test vector for agency predictor:  (5866, 4075)


# Naive Bayes Implementation

In [0]:
# Prior probabilities for the classes
def class_probability(data):
  example_numb = len(data)
  prob = dict(Counter(data))
  for key in prob.keys():
    prob[key] = prob[key] / float(example_numb)
  return prob

In [0]:
def feature_probability(data, sum_of_words, V):
  # Calculating the sum of all weights (Count in CountVectorizer or TF-IDF value in TfidfVectorzier)
  sum_of_weights = 0.0
  for w in data:
    sum_of_weights += w
  # Calculating probabilty of this feature, using Add 1 Smoothing 
  p = (sum_of_weights + 1) / (sum_of_words + V)
  return p

In [0]:
def naive_bayes_train(x, y, voc):
  # x = x_train_vec, y = y_train, voc = vocab
  
  # Calculating probability of each class
  y_probability = class_probability(y)
  
  # List of all unique class
  y_classes = np.unique(y)
  print ('This model have %d class' % len(y_classes))
  
  # Shape of input
  x_rows, x_columns = x.shape
  print ('This model data have %d rows and %d columns' % (x_rows, x_columns))
  
  # Probabilites of each feature based on each class
  likelihoods = {}

  # Initialize
  for cls in y_classes:
    likelihoods[cls] = defaultdict(list)

  for cls in y_classes:
    # For every class
    # Find which doument category (y_train) is same as current class
    row_indices = np.where(y == cls)[0]
    
    # Get the documents (sentences vector)
    docs = x[row_indices, :]
    
    # Calculating all word weights
    sum_of_words = 0
    for d in docs:
      for v in d:
        sum_of_words += v
        
    # Remember the shape of this class (number of document and the length of vectors)
    doc_numb, vec_len  = np.shape(docs)
    
    # For every feature (word), add it's value to current class
    for i in range(0, vec_len):
      likelihoods[cls][i] += list(docs[:,i])
      
    # Calculate probablity of every feature given this class
    for i in range(0, vec_len):
      likelihoods[cls][i] = feature_probability(likelihoods[cls][i], sum_of_words, len(voc))
      
  return y_probability, likelihoods
  

In [0]:
def naive_bayes_test(x, y, class_prob, features_prob):
  
  prediction = {}
  y_list = list(y)
  
  # For every test data
  for i in range(0, len(x)):
    current_x = x[i]
    current_y = y_list[i]
    
    # A dictionary that contains prediction value based on each class
    current_prediction = {}
    for cls in class_prob.items():
      
      predict = 1
      
      # Multiply by probability of current class
      current_cls_index = cls[0]
      current_cls_value = cls[1]
      predict = predict * current_cls_value
      
      # For each feature (word) get the probability based on current class
      for index, word in enumerate(current_x):
        if word > 0 :
          word_probability = (features_prob[current_cls_index][index] ** word)
          predict = predict * word_probability
          
      # Add new predict to prediction list of this test data
      current_prediction[current_cls_index] = predict
      
    # Sort prediction
    sorted_by_value = sorted(current_prediction.items(), key=lambda kv: kv[1], reverse=True)
    
    # Add prediction list of this test data to global predictation list
    prediction[i] = sorted_by_value
    
  return prediction

In [0]:
def get_labels_of_prediction(pred, numb):
  result = {}
  for index, data in enumerate(pred.items()):
    labels = []
    for l in data[1]:
      labels.append(l[0])
    if numb == 'All':
      result[index] = labels
    else:
      result[index] = labels[0:numb]
  return result

## Naive Bayes as Category Predictor

In [35]:
p_class, p_features = naive_bayes_train(x_cat_train_vec, y_cat_train, cat_vocab)
# p_class is probability of each class
# print(p_class)
# p_features consist of probability of each feature based on each class
# print(p_features)

This model have 20 class
This model data have 11461 rows and 3934 columns


In [0]:
# Make prediction
prediction = naive_bayes_test(x_cat_test_vec, y_cat_test, p_class, p_features)
# print(prediction)

# Get the top N class (here is 1)
prediction_labels = get_labels_of_prediction(prediction, 1)
# print(prediction_labels)

# Convert to a single value list
y_cat_predicted = []
for p in prediction_labels.items():
  y_cat_predicted.append(p[1][0])
# print(y_cat_predicted)

# Convert real y_test to a list
y_cat_real = list(y_cat_test)
# print(y_cat_real)

In [37]:
print('Accuracy Score:', accuracy_score(y_cat_real, y_cat_predicted))
print('Precision Score:', precision_score(y_cat_real, y_cat_predicted, average='macro'))
print('Recall Score:', recall_score(y_cat_real, y_cat_predicted, average='macro'))
print('F1 Score:', f1_score(y_cat_real, y_cat_predicted, average='macro'))

Accuracy Score: 0.7173761339846476
Precision Score: 0.4017680968427543
Recall Score: 0.31785185079867506
F1 Score: 0.3206821375907375


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Naive Bayes as Agency Predictor

In [38]:
p_class_ag, p_features_ag = naive_bayes_train(x_ag_train_vec, y_ag_train, ag_vocab)
# p_class is probability of each class
# print(p_class)
# p_features consist of probability of each feature based on each class
# print(p_features)

This model have 2 class
This model data have 23461 rows and 4075 columns


In [0]:
# Make prediction
prediction = naive_bayes_test(x_ag_test_vec, y_ag_test, p_class_ag, p_features_ag)
# print(prediction)

# Get the top N class (here is 1)
prediction_labels = get_labels_of_prediction(prediction, 1)
# print(prediction_labels)

# Convert to a single value list
y_ag_predicted = []
for p in prediction_labels.items():
  y_ag_predicted.append(p[1][0])
# print(y_cat_predicted)

# Convert real y_test to a list
y_ag_real = list(y_ag_test)
# print(y_cat_real)

In [40]:
print('Accuracy Score:', accuracy_score(y_ag_real, y_ag_predicted))
print('Precision Score:', precision_score(y_ag_real, y_ag_predicted, average='macro'))
print('Recall Score:', recall_score(y_ag_real, y_ag_predicted, average='macro'))
print('F1 Score:', f1_score(y_ag_real, y_ag_predicted, average='macro'))

Accuracy Score: 0.9152744630071599
Precision Score: 0.9172874059721463
Recall Score: 0.9136649154865235
F1 Score: 0.9147570866638908


# Extra

## Baseline Models (Sklearn library)

In [46]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline

# When building the vocabulary ignore terms that have a document frequency strictly lower than
# the given threshold. This value is also called cut-off in the literature.
min_df = 0.005


# Tokenize function used in Vectorizer
def tokenize(text):
    return word_tokenize(text)


# (Multinomial) Naive Bayes Model
text_clf = Pipeline([('vect', CountVectorizer(tokenizer=tokenize,
                                              analyzer='word', ngram_range=(1, 2), min_df=min_df, lowercase=False)),
                     ('tfidf', TfidfTransformer(sublinear_tf=True)),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(x_cat_train_corpus, y_cat_train)
naive_score = text_clf.score(x_cat_test_corpus, y_cat_test)
print('Naive Bayes Model: ', naive_score)
# SGD (Stochastic Gradient Descent) Model
text_clf_sgd = Pipeline([('vect', CountVectorizer(tokenizer=tokenize,
                                                  analyzer='word', ngram_range=(1, 2), min_df=min_df, lowercase=False)),
                         ('tfidf', TfidfTransformer(sublinear_tf=True)),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                   alpha=1e-3, max_iter=5))])
text_clf_sgd = text_clf_sgd.fit(x_cat_train_corpus, y_cat_train)
sgd_score = text_clf_sgd.score(x_cat_test_corpus, y_cat_test)
print('SGD Model: ', sgd_score)

# Linear Support Vector Machine Model
text_clf_linear_svc = Pipeline([('vect', CountVectorizer(tokenizer=tokenize,
                                                         analyzer='word', ngram_range=(1, 2),
                                                         min_df=min_df, lowercase=False)),
                                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                                ('clf-svm', LinearSVC(loss='hinge', penalty='l2',
                                                      max_iter=5))])

text_clf_linear_svc = text_clf_linear_svc.fit(x_cat_train_corpus, y_cat_train)
linear_svc_score = text_clf_linear_svc.score(x_cat_test_corpus, y_cat_test)
print('Linear SVC Model: ', linear_svc_score)

Naive Bayes Model:  0.7194696441032798


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGD Model:  0.717725052337753


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Linear SVC Model:  0.7386601535240753


## Neural Network Model (Keras Bidirectional LSTM)

In [0]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Convolution1D
from keras.layers import CuDNNLSTM
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.models import Model
from keras import optimizers
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy
from keras.utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
import codecs

In [0]:
lstm_x_train = x_cat_train['title'].values
lstm_x_train = preprocess(lstm_x_train)
# test set
lstm_x_test = x_cat_test['title'].values
lstm_x_test = preprocess(lstm_x_test)

In [0]:
num_words = 10000

# create the tokenizer
tokenizer = Tokenizer(num_words=num_words)

# fit the tokenizer on the documents
tokenizer.fit_on_texts(x_cat_train_corpus)

# sequence encode
encoded_docs = tokenizer.texts_to_sequences(lstm_x_train)

# pad sequences
max_length = max([len(s.split()) for s in lstm_x_train])
x_train_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

encoded_docs = tokenizer.texts_to_sequences(lstm_x_test)
x_test_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [0]:
y_cat_train_classes = np.unique(y_cat_train)
y_cat_train_classes_len = len(y_cat_train_classes)

y_cat_test_classes = np.unique(y_cat_train)
y_cat_test_classes_len = len(y_cat_test_classes)

In [159]:
categorical_y_train = to_categorical(y_cat_train, y_cat_train_classes_len)
categorical_y_test = to_categorical(y_cat_test, y_cat_test_classes_len)

model = Sequential()
model.add(Embedding(vocab_size, 200, input_length=max_length))
model.add(Bidirectional(CuDNNLSTM(100, return_sequences=True, name='lstm_layer')))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(200, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(y_cat_train_classes_len, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[categorical_accuracy])

model.summary()
batch_size = 8
epochs = 4
hist = model.fit(x_train_padded, categorical_y_train, batch_size=batch_size, epochs=epochs)

loss, acc = model.evaluate(x_test_padded, categorical_y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 22, 200)           16597200  
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 22, 200)           241600    
_________________________________________________________________
global_max_pooling1d_28 (Glo (None, 200)               0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_55 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_58 (Dense)             (None, 20)                4020      
Total para